In [1]:
#importing libs requerid
from urllib.request import urlopen
from bs4 import BeautifulSoup
from requests import get
from urllib.error import HTTPError 
from urllib.error import URLError
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random

sns.set()

In [10]:
#setting up the lists that will form the dataframe with all the results

adress = []
prices = []
cond = []
iptu = []
areas = []
bathrooms = []
rooms = []
descriptions = []
date = []
urls = []

In [12]:
%%time
#page inicial
n_pages = 0
rows = 0 
for page in range(0,8):
    
    n_pages += 1
    
    #loading url
    zap_url = "https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?__zt=rpe%3Aa&onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro,-22.906846,-43.172896&pagina="+str(n_pages)+"&tipo=Im%C3%B3vel%20usado&transacao=Venda"
    headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
    response = get(zap_url, headers=headers)
    
    #beatifulSoup
    bs = BeautifulSoup(response.text, 'html.parser')
    house_containers = bs.find_all('div', class_="card-container")

    for container in house_containers:
        

        #Get adress
        lista_adress = container.find_all(class_='simple-card__address')
        adress.append(lista_adress[0].text)

        #Get price
        lista_price = container.find_all('strong')
        var_1 = lista_price[0].text.replace('\n','')
        var_2 = var_1.replace('R$', '')
        var_3 = var_2.replace('.', '').strip()
        if var_3 != ('Sob consulta'):
            var_4 = float(var_3)
            prices.append(var_4)
        else:
            prices.append(None)

        #Get cond
        lista_cond = container.find_all(class_='condominium')
        if len(lista_cond) != 0:
            var_1 = (lista_cond[0].text).replace('condomínio','').replace('.','')
            var_1  = float(var_1.replace('/n', '').replace('R$','').strip())
            cond.append(var_1)
        else:
            cond.append(None)
        
        #Get IPTU
        lista_iptu = container.find_all(class_='iptu')
        if len(lista_iptu) != 0:
            var_1 = (lista_iptu[0].text).replace('IPTU','').replace('.','')
            var_1  = float(var_1.replace('/n', '').replace('R$','').strip())
            iptu.append(var_1)
        else:
            iptu.append(None)

        #Get areas
        lista_areas = container.find_all(class_='js-areas')
        if len(lista_areas) != 0:
            var_1 = lista_areas[0].text.replace('/n', '').strip()
            areas.append(var_1)
        else:
            areas.append(None)

        #Get bathrooms
        lista_bath = container.find_all(class_='js-bathrooms')
        if len(lista_bath) != 0:
            var_1 = lista_bath[0].text.replace('/n', '').strip()
            bathrooms.append(var_1)
        else:
            bathrooms.append(None)

        #Get Rooms
        lista_room = container.find_all(class_='js-bedrooms')
        if len(lista_room) != 0:
            var_1 = lista_room[0].text.replace('/n', '').strip()
            rooms.append(var_1)
        else:
            rooms.append(None)

        #Get Description
        lista_desc = container.find_all(class_='collapse simple-card__description')
        if len(lista_desc) != 0:
            var_1 = lista_desc[0].text.replace('/n', '').replace('\n','').strip()
            descriptions.append(var_1)
        else:
            descriptions.append(None)
        
        rows += 1
            
        time.sleep(1)
    
    print('Scraped {} pages containing {} properties.'.format(n_pages, rows))


You scraped 1 pages containing 24 properties.
You scraped 2 pages containing 48 properties.
You scraped 3 pages containing 72 properties.
You scraped 4 pages containing 96 properties.
You scraped 5 pages containing 120 properties.
You scraped 6 pages containing 144 properties.
You scraped 7 pages containing 168 properties.
You scraped 8 pages containing 192 properties.
Wall time: 3min 28s


In [4]:
print(len(adress))
print(len(prices))
print(len(cond))
print(len(iptu))
print(len(areas))
print(len(bathrooms))
print(len(rooms))
print(len(descriptions))

600
600
600
600
600
600
600
600


In [5]:
#columns of the dataframe
cols = ['Adress', 'Price', 'Cond (R$)', 'IPTU (R$)', 'Size (m²)', 'Bathrooms', 'Rooms', 'Description']

#Dataframe
riodejaneiro = pd.DataFrame({'Adress': adress,
                             'Price': prices,
                             'Cond (R$)': cond,
                             'IPTU (R$)': iptu,
                             'Size (m²)': areas,
                             'Bathrooms': bathrooms,
                             'Rooms': rooms,
                             'Description': descriptions,
                            })[cols]

In [6]:
riodejaneiro

,Adress,Price,Cond (R$),IPTU (R$),Size (m²),Bathrooms,Rooms,Description
0,"Rua da Assembléia, Centro",388900.0,NaN,NaN,27 - 408 m²,None,None,Invista no último terreno na Rua da Assembleia...
1,"Rua das Marrecas, Centro",NaN,NaN,NaN,33 - 70 m²,None,1,Hub Coliving. Studios com área de convivência ...
2,"Rua dos Inválidos, Centro",180700.0,NaN,NaN,20 - 77 m²,None,None,"Na Lapa, próximo ao metrô, as barcas e a centr..."
3,"Avenida Vieira Souto, Ipanema",12400000.0,3300.0,2252.0,210 m²,4,3,O apartamento possui boa distribuição dos ambi...
4,"Rua Antônio Basílio, Tijuca",480000.0,987.0,158.0,86 m²,2,2,"Imóvel em excelente localização, sol matinal, ..."
...,...,...,...,...,...,...,...,...
595,"Rua Barão do Bom Retiro, Engenho Novo",270000.0,NaN,524.0,108 m²,3,3,Casa localizada próximo ao Grajaú. Transporte ...
596,"Estrada Coronel Pedro Corrêa, Jacarepaguá",375000.0,600.0,NaN,66 m²,2,2,"Apartamento novo, primeira utilização, no préd..."
597,"Rua Turvânia, Vila Valqueire",490000.0,300.0,1170.0,111 m²,3,3,Vendo ou alugo excelente apartamento localizad...
598,"Rua Piauí, Todos os Santos",265000.0,550.0,248.0,55 m²,2,2,Lindo apartamento no Condomínio Boa Nova Resid...


In [9]:
#exportar o dataframe para o excel
escrever = pd.ExcelWriter('rio_raw.xls')
riodejaneiro.to_excel(escrever, 'riosheet')
escrever.save()

In [ ]:
try:
    html = urlopen("https://pythonscrapingthisurldoesnotexist.com")
except HTTPError as e:
    print("The server returned an HTTP error")
except URLError as e:
    print("The server could not be found!")
else:
    print(html.read())

In [ ]:
def getTitle(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    try:
        bsObj = BeautifulSoup(html.read(), "lxml")
        title = bsObj.body.h1
    except AttributeError as e:
        return None
    return title


title = getTitle("http://www.pythonscraping.com/pages/page1.html")
if title == None:
    print("Title could not be found")
else:
    print(title)